In [1]:
import math
import numpy as np
import pandas as pd
from oucp_utils import *
from scipy.optimize import fsolve
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

In [2]:
# carbon content per unit of energy
h1 = 1

# elasticity parameters
sigma = 1
sigmaE = 1
theta = 4
rho = 0
alpha = 0.15

scenario = 'opt'

# elasticity of supply of renewables
epsilonSstar1 = 0.5
epsilonS1 = 0.5

# baseline, no renewable energy
epsilonSvec = [(epsilonS1, h1, 1)]
epsilonSstarvec = [(epsilonSstar1, h1, 1)]
assert(sum(k for i,j,k in epsilonSvec) == 1)
assert(sum(k for i,j,k in epsilonSstarvec) == 1)


data = pd.read_excel("../../raw_data/baselinecarbon_direct_2018.xlsx")
#data = pd.read_csv("../../raw_data/BaselineCarbon_2015.csv")
#data['Cey'] = data['Cey'] + data['Ced']
#data['Ceystar'] = data['Ceystar'] + data['Cedstar']
#data['Ced'] = 0
#data['Cedstar'] = 0
data['jxbar']=data['Cex']/(data['Cex'] + data['Ceystar'])
data['jmbar']=data['Cey']/(data['Cey'] + data['Cem'])

if scenario == 'constrained':
    data = data[(data['region_scenario'] == 3)] 
    tax_scenario= ['purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid']
elif scenario == 'global':
    data = data[(data['region_scenario'] == 3)] 
    tax_scenario= ['global']
elif scenario == 'opt':
    data = data[data['region_scenario'] != 4]
    tax_scenario= ['Unilateral']
#tax_scenario = ['Unilateral']
data = data[(data['region_scenario'] == 3)]
tax_scenario = ['EPC_hybrid']

In [3]:
def max_welfare(p, phi, tax, region_data):
    return -model.comp_welfare(p, phi, tax,region_data)

In [4]:
phi_list = np.arange(0,2,0.1)
model_parameters = (theta, sigma, sigmaE, epsilonSvec, epsilonSstarvec, rho, alpha)
model = taxModel(data, tax_scenario, phi_list, model_parameters)

In [25]:
index, region_data = next(model.data.iterrows())
tax = 'Unilateral'
phi = 2

In [ ]:
res = minimize(max_welfare, [1, 0], args = (phi, tax, region_data), method='nelder-mead', tol=1e-8, options={'maxfev': 1000}, bounds = [(0.01,np.inf), (0, 0)])
res

In [27]:
res = minimize(max_welfare, [1, 0], args = (phi, tax, region_data), method='trust-constr', tol=1e-8, bounds = [(0.01,np.inf), (0, 0)])
res

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 42
      cg_stop_cond: 1
            constr: [array([0.83855744, 0.        ])]
       constr_nfev: [0]
       constr_nhev: [0]
       constr_njev: [0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 2.437734842300415
               fun: -10.647481211506419
              grad: array([3.57627869e-07, 0.00000000e+00])
               jac: [<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>]
   lagrangian_grad: array([1.44569362e-07, 0.00000000e+00])
           message: '`xtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 57
              nhev: 0
               nit: 54
             niter: 54
              njev: 19
        optimality: 1.4456936174804319e-07
            status: 2
           success: True
         tr_radius: 1.0000000000000005e-09
               

In [30]:
pe = fsolve(model.comp_cons_eq, [1], args = (2, res.x, phi, tax, region_data))[0]
pe

1.0752806202030305

In [31]:
arr = [pe, res.x[0], res.x[1]]
model.comp_all(pe, 2, arr[1:3], phi, tax, region_data)['welfare']

10.647481211506419

In [32]:
model.comp_obj(arr[0], 2, arr[1:3], phi, tax, region_data)

(0.0, -0.00167881738659403, 0)

In [35]:
pe - 1.0752739832662797

6.636936750847866e-06

In [36]:
res.x - [0.83860897,0]

array([-5.15326302e-05,  0.00000000e+00])

In [33]:
model.comp_obj(1.0752739832662797, 2, [0.83860897,0], phi, tax, region_data)

(-3.552713678800501e-13, 1.32582211875842e-7, 0)

In [34]:
model.comp_all(1.0752739832662797,2, [0.83860897,0], phi, tax, region_data)['welfare']

10.647481204808946

In [14]:
model.comp_obj(1.05739466576942, 2, [0.8465, 0], phi, tax, region_data)

(-0.5452638053042485, 0.606415162319092, 0)

In [16]:
def obj_fn_te(p, phi, tax, region_data):
    diff, diff1 = model.te_obj(p, phi, tax, region_data)
    return abs(diff) + abs(diff1)

def obj_fn(p, phi, tax, region_data):
    diff, diff1, diff2 = model.obj_system(p, phi, tax, region_data)
    return abs(diff) + abs(diff1) + abs(diff2)

In [21]:
model.solveOne(phi,tax,region_data, 1,0,2,1)

did not converge, tax is PC_hybrid region is OECD37 as Home phi is 2 guess is [1, 0, 2]
converged on second try


(0.8566417266416129,
 0.8158650732485632,
 0.5275261066696807,
 0.8158650732485632,
 1)

In [10]:
init_guess = np.random.rand(1000, 2) * 10000
result = []
for pe, tb in init_guess:
    #res = model.solve_obj(phi, tax, region_data, init_guess = [pe,tb, 0], verbose = False, second_try = False)
    res = minimize(obj_fn, [pe,tb], args = (phi, tax, region_data), method='nelder-mead', tol=0.000001, options={'maxfev': 100000})
    result.append((res.x,res.fun, (pe,tb)))

In [13]:
for res in result:
    if not (np.isclose(abs(res[0]), [1.24094625,1.33333333])).all():
        print(res)
    #if res[1] == 1:
    #    print(res)

(array([ 1.09860833, -0.79654207]), 26.688391712445423, (5.699311173572585, 5093.960546141944))


In [15]:
#model.solve_obj(2, tax, region_data, init_guess = [1,0,0], verbose = False)
fsolve(model.te_obj, [1,2], args = (phi, tax, region_data), full_output = True, maxfev = 100000)

(array([1.24094625, 1.33333333]),
 {'nfev': 10,
  'fjac': array([[-0.92146996,  0.38844962],
         [-0.38844962, -0.92146996]]),
  'r': array([-44.30783921, -21.44420869,  30.04099335]),
  'qtf': array([7.19505380e-08, 9.34561708e-08]),
  'fvec': array([-7.13491488e-11, -4.09308143e-11])},
 1,
 'The solution converged.')